quick branch to extract col and row in addition to target/non-target

In [1]:
%matplotlib widget 
#widget
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
from IPython.display import display
import os
from help_func_new_paradigm import *

In [2]:
from help_func_new_paradigm import *

In [3]:
method= "A1"
feature_types=["B3"]

In [4]:
# Load the signal recording data
base_path = Path.cwd().parent  / "data"/ "ali_paradigm_training" 
df_eeg= pd.read_csv(base_path /'19.07.2.csv')
df_annotated = annotate_flash_and_target_columns(df_eeg)
df_with_trials = add_trial_count_column_for_flashes_only(df_annotated)
df_single = epoch_grouped_trialwise_average(df_with_trials, normalization=method, 
                                                        blink_threshold=120, blink_rejection=True)
                                                        
# Load the general data ; without filtering due to incomplete data
df_combined = process_and_combine_all_csvs(base_path,blink_rejection=False, normalization=method)

C:\Users\chris\AppData\Local\Temp\ipykernel_2688\3767611158.py:3: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg= pd.read_csv(base_path /'19.07.2.csv')
c:\Users\chris\Documents\Master_MSNE\SS25\BSPM\Biosignals\ML_try_Ege\help_func_new_paradigm.py:313: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg = pd.read_csv(file)
c:\Users\chris\Documents\Master_MSNE\SS25\BSPM\Biosignals\ML_try_Ege\help_func_new_paradigm.py:313: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg = pd.read_csv(file)
c:\Users\chris\Documents\Master_MSNE\SS25\BSPM\Biosignals\ML_try_Ege\help_func_new_paradigm.py:313: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg = pd.read_csv(file)
c:\Users\chris\Documents\Master_MSNE\SS25\BSPM\Biosignals\ML_try_Ege\he

## All Record Combined


In [5]:
from help_fischer import *

In [6]:
X, y = extract_features_from_df_combined(df_combined, fs=128, feature_types = feature_types)
print(X.shape, y.shape)

(300, 28) (300,)


In [7]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Simulate fold-wise 5-fold data (5 folds, 60 rows each, 6 rows per trial → 10 trials per fold)
# Each 6-row group represents one trial (first 3: columns, next 3: rows)
num_folds = 5
rows_per_fold = 60

# Session splits
session_splits = {
    0: np.arange(0, 60),    
    1: np.arange(60, 120),   
    2: np.arange(120, 180),
    3: np.arange(180, 240),    
    4: np.arange(240, 300)    
}

results = []
num_folds = 5

for test_fold in range(num_folds):
    test_trials = session_splits[test_fold]
    train_trials = np.setdiff1d(np.arange(300), test_trials)

    X_train_full, X_test_full = X[train_trials], X[test_trials]
    y_train, y_test = y[train_trials], y[test_trials]

    # === Feature selection ===
    X_train, selected_idx = select_top_k_fisher_multiclass(X_train_full, y_train, k=10)
    X_test = X_test_full[:, selected_idx]

    # === Feature Scaling ===
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # === Train classifier ===
    clf = SVC(kernel='linear', probability=True, random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_probs = clf.predict_proba(X_test_scaled)[:, 1]

    # === WTA evaluation ===
    correct_combined = 0
    correct_column = 0
    correct_row = 0

    for i in range(0, len(y_test), 6):
        block_probs = y_probs[i:i+6]
        block_y = y_test[i:i+6]

        top_col = np.argmax(block_probs[0:3])
        top_row = np.argmax(block_probs[3:6])

        correct_col = np.argmax(block_y[0:3])
        correct_row_val = np.argmax(block_y[3:6])

        correct_column += (top_col == correct_col)
        correct_row += (top_row == correct_row_val)
        correct_combined += ((top_col == correct_col) and (top_row == correct_row_val))

    fold_result = {
        "Fold": test_fold,
        "Accuracy_Combined": correct_combined / (len(y_test) // 6),
        "Accuracy_Column": correct_column / (len(y_test) // 6),
        "Accuracy_Row": correct_row / (len(y_test) // 6),
    }

    results.append(fold_result)

df_results = pd.DataFrame(results)
mean_row = df_results.mean(numeric_only=True)
mean_row["Fold"] = "Mean"
std_row = df_results.std(numeric_only=True)
std_row["Fold"] = "Std"
df_results = pd.concat([df_results, pd.DataFrame([mean_row, std_row])], ignore_index=True)
display(df_results)

C:\Users\chris\AppData\Local\Temp\ipykernel_2688\4010560686.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Mean' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mean_row["Fold"] = "Mean"
C:\Users\chris\AppData\Local\Temp\ipykernel_2688\4010560686.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Std' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  std_row["Fold"] = "Std"


,Fold,Accuracy_Combined,Accuracy_Column,Accuracy_Row
0,0,0.000000,0.100000,0.200000
1,1,0.200000,0.500000,0.400000
2,2,0.300000,0.400000,0.600000
3,3,0.000000,0.100000,0.400000
4,4,0.200000,0.400000,0.300000
5,Mean,0.140000,0.300000,0.380000
6,Std,0.134164,0.187083,0.148324


### Selected electrodes - Didn't work

In [8]:
selected_idx = [3, 11, 24, 26, 27]
# Session splits
session_splits = {
    0: np.arange(0, 60),    
    1: np.arange(60, 120),   
    2: np.arange(120, 180),
    3: np.arange(180, 240),    
    4: np.arange(240, 300)    
}

results = []
num_folds = 5

for test_fold in range(num_folds):
    test_trials = session_splits[test_fold]
    train_trials = np.setdiff1d(np.arange(300), test_trials)

    X_train_full, X_test_full = X[train_trials], X[test_trials]
    y_train, y_test = y[train_trials], y[test_trials]

    # === Feature selection ===
    X_train = X_train_full[:, selected_idx]
    X_test = X_test_full[:, selected_idx]

    # === Feature Scaling ===
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # === Train classifier ===
    clf = SVC(kernel='linear', probability=True, random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_probs = clf.predict_proba(X_test_scaled)[:, 1]

    # === WTA evaluation ===
    correct_combined = 0
    correct_column = 0
    correct_row = 0

    for i in range(0, len(y_test), 6):
        block_probs = y_probs[i:i+6]
        block_y = y_test[i:i+6]

        top_col = np.argmax(block_probs[0:3])
        top_row = np.argmax(block_probs[3:6])

        correct_col = np.argmax(block_y[0:3])
        correct_row_val = np.argmax(block_y[3:6])

        correct_column += (top_col == correct_col)
        correct_row += (top_row == correct_row_val)
        correct_combined += ((top_col == correct_col) and (top_row == correct_row_val))

    fold_result = {
        "Fold": test_fold,
        "Accuracy_Combined": correct_combined / (len(y_test) // 6),
        "Accuracy_Column": correct_column / (len(y_test) // 6),
        "Accuracy_Row": correct_row / (len(y_test) // 6),
    }

    results.append(fold_result)

df_results = pd.DataFrame(results)
mean_row = df_results.mean(numeric_only=True)
mean_row["Fold"] = "Mean"
std_row = df_results.std(numeric_only=True)
std_row["Fold"] = "Std"
df_results = pd.concat([df_results, pd.DataFrame([mean_row, std_row])], ignore_index=True)
display(df_results)

C:\Users\chris\AppData\Local\Temp\ipykernel_2688\2651718423.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Mean' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mean_row["Fold"] = "Mean"
C:\Users\chris\AppData\Local\Temp\ipykernel_2688\2651718423.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Std' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  std_row["Fold"] = "Std"


,Fold,Accuracy_Combined,Accuracy_Column,Accuracy_Row
0,0,0.100000,0.200000,0.300000
1,1,0.100000,0.300000,0.300000
2,2,0.100000,0.300000,0.500000
3,3,0.000000,0.700000,0.200000
4,4,0.100000,0.300000,0.400000
5,Mean,0.080000,0.360000,0.340000
6,Std,0.044721,0.194936,0.114018


## 1 Record Only

In [9]:
X, y = extract_features_from_df_combined(df_combined, fs=128, feature_types = feature_types)
print(X.shape, y.shape)

(300, 28) (300,)


In [10]:
num_folds = 2
rows_per_fold = 12

# Session splits
session_splits = {
    0: np.arange(0, 12),    
    1: np.arange(12, 24),   
    2: np.arange(24, 36),
    3: np.arange(36, 48),    
    4: np.arange(48, 60)    
}

results = []
num_folds = 5

for test_fold in range(num_folds):
    test_trials = session_splits[test_fold]
    train_trials = np.setdiff1d(np.arange(60), test_trials)

    X_train_full, X_test_full = X[train_trials], X[test_trials]
    y_train, y_test = y[train_trials], y[test_trials]

    # === Feature selection ===
    X_train, selected_idx = select_top_k_fisher_multiclass(X_train_full, y_train, k=10)
    X_test = X_test_full[:, selected_idx]

    # === Feature Scaling ===
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # === Train classifier ===
    clf = SVC(kernel='linear', probability=True, random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_probs = clf.predict_proba(X_test_scaled)[:, 1]

    # === WTA evaluation ===
    correct_combined = 0
    correct_column = 0
    correct_row = 0

    for i in range(0, len(y_test), 6):
        block_probs = y_probs[i:i+6]
        block_y = y_test[i:i+6]

        top_col = np.argmax(block_probs[0:3])
        top_row = np.argmax(block_probs[3:6])

        correct_col = np.argmax(block_y[0:3])
        correct_row_val = np.argmax(block_y[3:6])

        correct_column += (top_col == correct_col)
        correct_row += (top_row == correct_row_val)
        correct_combined += ((top_col == correct_col) and (top_row == correct_row_val))

    fold_result = {
        "Fold": test_fold,
        "Accuracy_Combined": correct_combined / (len(y_test) // 6),
        "Accuracy_Column": correct_column / (len(y_test) // 6),
        "Accuracy_Row": correct_row / (len(y_test) // 6),
    }

    results.append(fold_result)

df_results = pd.DataFrame(results)
mean_row = df_results.mean(numeric_only=True)
mean_row["Fold"] = "Mean"
std_row = df_results.std(numeric_only=True)
std_row["Fold"] = "Std"
df_results = pd.concat([df_results, pd.DataFrame([mean_row, std_row])], ignore_index=True)
display(df_results)

C:\Users\chris\AppData\Local\Temp\ipykernel_2688\2169775392.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Mean' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mean_row["Fold"] = "Mean"
C:\Users\chris\AppData\Local\Temp\ipykernel_2688\2169775392.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Std' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  std_row["Fold"] = "Std"


,Fold,Accuracy_Combined,Accuracy_Column,Accuracy_Row
0,0,0.000000,0.00000,0.500000
1,1,0.000000,0.50000,0.000000
2,2,0.000000,0.00000,0.000000
3,3,0.500000,1.00000,0.500000
4,4,0.500000,0.50000,0.500000
5,Mean,0.200000,0.40000,0.300000
6,Std,0.273861,0.41833,0.273861


In [11]:
selected_idx = [3, 11, 24, 26, 27]
# Session splits
session_splits = {
    0: np.arange(0, 60),    
    1: np.arange(60, 120),   
    2: np.arange(120, 180),
    3: np.arange(180, 240),    
    4: np.arange(240, 300)    
}

results = []
num_folds = 5

for test_fold in range(num_folds):
    test_trials = session_splits[test_fold]
    train_trials = np.setdiff1d(np.arange(300), test_trials)

    X_train_full, X_test_full = X[train_trials], X[test_trials]
    y_train, y_test = y[train_trials], y[test_trials]

    # === Feature selection ===
    X_train = X_train_full[:, selected_idx]
    X_test = X_test_full[:, selected_idx]

    # === Feature Scaling ===
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # === Train classifier ===
    clf = SVC(kernel='linear', probability=True, random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_probs = clf.predict_proba(X_test_scaled)[:, 1]

    # === WTA evaluation ===
    correct_combined = 0
    correct_column = 0
    correct_row = 0

    for i in range(0, len(y_test), 6):
        block_probs = y_probs[i:i+6]
        block_y = y_test[i:i+6]

        top_col = np.argmax(block_probs[0:3])
        top_row = np.argmax(block_probs[3:6])

        correct_col = np.argmax(block_y[0:3])
        correct_row_val = np.argmax(block_y[3:6])

        correct_column += (top_col == correct_col)
        correct_row += (top_row == correct_row_val)
        correct_combined += ((top_col == correct_col) and (top_row == correct_row_val))

    fold_result = {
        "Fold": test_fold,
        "Accuracy_Combined": correct_combined / (len(y_test) // 6),
        "Accuracy_Column": correct_column / (len(y_test) // 6),
        "Accuracy_Row": correct_row / (len(y_test) // 6),
    }

    results.append(fold_result)

df_results = pd.DataFrame(results)
mean_row = df_results.mean(numeric_only=True)
mean_row["Fold"] = "Mean"
std_row = df_results.std(numeric_only=True)
std_row["Fold"] = "Std"
df_results = pd.concat([df_results, pd.DataFrame([mean_row, std_row])], ignore_index=True)
display(df_results)

C:\Users\chris\AppData\Local\Temp\ipykernel_2688\2651718423.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Mean' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mean_row["Fold"] = "Mean"
C:\Users\chris\AppData\Local\Temp\ipykernel_2688\2651718423.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Std' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  std_row["Fold"] = "Std"


,Fold,Accuracy_Combined,Accuracy_Column,Accuracy_Row
0,0,0.100000,0.200000,0.300000
1,1,0.100000,0.300000,0.300000
2,2,0.100000,0.300000,0.500000
3,3,0.000000,0.700000,0.200000
4,4,0.100000,0.300000,0.400000
5,Mean,0.080000,0.360000,0.340000
6,Std,0.044721,0.194936,0.114018


In [12]:
# Save SVC trained on all data

# === Feature Scaling ===
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# === Train classifier ===
clf = SVC(kernel='linear', probability=True, random_state=42)
clf.fit(X_scaled, y)
#y_probs = clf.predict_proba(X_test_scaled)[:, 1]


import joblib

# Save the classifier to a file
joblib.dump(clf, 'svm_classifier.pkl')




['svm_classifier.pkl']

# It Doesn't work for real time we can juse use 

In [13]:
# Load the signal recording data
base_path = Path.cwd().parent  / "data"/ "ali_paradigm_training"  # Whereever we upload for 1 trial   
method = ["A1"]                                                  
df_combined = process_and_combine_all_csvs(base_path,blink_rejection=False, normalization=method)
X, y = extract_features_from_df_combined(df_combined, fs=128, feature_types = feature_types)

c:\Users\chris\Documents\Master_MSNE\SS25\BSPM\Biosignals\ML_try_Ege\help_func_new_paradigm.py:313: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg = pd.read_csv(file)


ValueError: Invalid normalization type.